In [1]:
%reload_ext autoreload
%autoreload 2

import vtk
from vtk.util import numpy_support

import numpy as np
import bloscpack as bp
import time
import sys
import os
sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))

from utilities2015 import *
from annotation_utilities import *

import matplotlib.pyplot as plt
%matplotlib inline

from vis3d_utilities import *

/usr/lib/python2.7/dist-packages/pkg_resources.py:1031: UserWarning: /home/yuncong/.python-eggs is writable by group/others and vulnerable to attack when used with get_resource_filename. Consider a more secure location (set with .set_extraction_path or the PYTHON_EGG_CACHE environment variable).
  warnings.warn(msg, UserWarning)


In [2]:
volume_dir = '/home/yuncong/CSHL_volumes/'

In [7]:
stack = 'MD585'

In [ ]:
tb_volume = bp.unpack_ndarray_file(volume_dir + "/%(stack)s/%(stack)s_thumbnailVolume.bp" % {'stack': stack})

In [8]:
score_imagedata = volume_to_imagedata(tb_volume)

# Render volumes
# http://www.vtk.org/Wiki/VTK/Examples/Cxx/VolumeRendering/SmartVolumeMapper

ren1 = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren1)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# contour_mapper = vtk.vtkPolyDataMapper()
# contour_mapper.SetInputData(polydata_outerContour)

# contour_actor = vtk.vtkActor()
# contour_actor.SetMapper(contour_mapper)
# #     a.GetProperty().SetRepresentationToWireframe()
# contour_actor.GetProperty().SetColor(1,1,1)
# contour_actor.GetProperty().SetOpacity(.1)
# ren1.AddActor(contour_actor)

imageData = score_imagedata

volumeMapper = vtk.vtkSmartVolumeMapper()
#     volumeMapper.SetBlendModeToComposite()
volumeMapper.SetInputData(imageData)

volumeProperty = vtk.vtkVolumeProperty()
#     volumeProperty.ShadeOff()
# volumeProperty.SetInterpolationType(vtk.VTK_LINEAR_INTERPOLATION)

compositeOpacity = vtk.vtkPiecewiseFunction()
compositeOpacity.AddPoint(0.0, 0.0)
compositeOpacity.AddPoint(240., 0.1)
compositeOpacity.AddPoint(255.0, 0.1)
volumeProperty.SetScalarOpacity(compositeOpacity)

color = vtk.vtkColorTransferFunction()
color.AddRGBPoint(0.0, 0,0,0)
color.AddRGBPoint(200.0, .5,.5,.5)
color.AddRGBPoint(255.0, 1,1,1)
volumeProperty.SetColor(color)

volume = vtk.vtkVolume()
volume.SetMapper(volumeMapper)
volume.SetProperty(volumeProperty)

ren1.AddActor(volume)

camera = vtk.vtkCamera()

# 30 degree
camera.SetViewUp(0, -1, 0)
camera.SetPosition(-10, -5, -5)
camera.SetFocalPoint(1, 1, 1)

ren1.SetActiveCamera(camera)

ren1.ResetCamera()

axes = add_axes(iren)

renWin.Render()
renWin.SetWindowName('Thumbnail Volume %(stack)s' % {'stack': stack})
#     renWin.SetSize(600, 600)

# windowToImageFilter = vtk.vtkWindowToImageFilter()

# windowToImageFilter.SetInput(renWin);
# windowToImageFilter.SetMagnification(3); 
# windowToImageFilter.SetInputBufferTypeToRGBA();
# windowToImageFilter.ReadFrontBufferOff();
# windowToImageFilter.Update();

# writer = vtk.vtkPNGWriter()
# writer.SetFileName(volume_dir + "/%(stack)s/scoreVolume_screenshots/%(stack)s_scoreVolume_%(name)s.png" % \
#                    {'stack':stack, 'name':name});
# writer.SetInputConnection(windowToImageFilter.GetOutputPort());
# writer.Write();

iren.Start()